<a href="https://colab.research.google.com/github/dudaholandah/PIBIC/blob/main/PIBIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install unidecode
!pip install umap-learn[plot]
!pip install trimap

In [164]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import plotly.express as px
from sklearn.manifold import TSNE
import re
import umap
import trimap
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import Isomap
from sklearn.manifold import trustworthiness
import plotly.graph_objects as go
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

# Pre-Processing Data

In [72]:
data = pd.read_excel("table_veg.xlsx", sheet_name='Veganos')

In [73]:
data.head()

,Classification,Product's Name,Ingredients,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,CARNES 2,Carne Moída do Futuro,"water, texturizedsoyprotein, peaprotein e chic...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,CARNES 3,Carne Moída do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,FRANGOS 3,Frango do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,SUÍNOS 1,Linguiça do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",50.0,139.0,7.7,0.0,5.7,9.5,3.9,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,CARNES 1,Hamburguer de Soja Goshen,"texturizedsoyprotein, water, soyoil, modified...",50.0,118.0,2.8,0.0,7.0,9.0,1.3,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Separating data

In [74]:
data_ingredients = pd.DataFrame(data['Ingredients'])
data_nutrients = data.drop(columns=['Ingredients', 'Classification', "Product's Name"])
data_clasification = pd.DataFrame(data['Classification'])
data_name = pd.DataFrame(data["Product's Name"])
label_data = data_clasification.join(data_name)

label_data.head()

,Classification,Product's Name
0,CARNES 2,Carne Moída do Futuro
1,CARNES 3,Carne Moída do Futuro
2,FRANGOS 3,Frango do Futuro
3,SUÍNOS 1,Linguiça do Futuro
4,CARNES 1,Hamburguer de Soja Goshen


In [75]:
data_nutrients.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,50.0,139.0,7.7,0.0,5.7,9.5,3.9,0.2,294.0,0.0,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,50.0,118.0,2.8,0.0,7.0,9.0,1.3,2.0,344.0,0.0,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Normalizing Nutrient Values (MinMax Scaler) 

In [76]:
X = data_nutrients.values
print(X)

[[ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 ...
 [ 40.   84.    5.2 ...   0.    0.    0. ]
 [ 10.   25.    3.3 ...   0.    0.    0. ]
 [  7.   28.    1.2 ...   0.    0.    0. ]]


In [77]:
attributes_dummies = data_nutrients.columns
normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

nutrients_normalized = pd.DataFrame(xscaled,columns=attributes_dummies)
nutrients_normalized = nutrients_normalized.replace(np.nan,0)

nutrients_normalized.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,0.125364,0.212121,0.171111,0.0,0.135714,0.193878,0.211957,0.002128,0.310782,0.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,0.125364,0.174688,0.062222,0.0,0.166667,0.183673,0.070652,0.021277,0.363636,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


### Normalizing Ingredients (One Hot Encode)

In [78]:
ingredients_normalized = pd.DataFrame()

for i in range(276):
  aux = data_ingredients.at[i,'Ingredients'].split(",")
  for w in aux:
    w = unidecode(w.lower().lstrip().replace('.',''))
    ingredients_normalized.at[i,w] = 1

ingredients_normalized = ingredients_normalized.replace(np.nan,0)
ingredients_normalized.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,cloreto de potassio,calciumdisodium and citricacid,isolatedvegetableprotein,hydrolyzedvegetableprotein,soy,potassiumsorbate,peastarch,peaprotein and sodiumbicarbonate,isolatedriceprotein,chickpeas and goldenflaxseed
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing data

In [79]:
raw_data = pd.DataFrame(ingredients_normalized)
raw_data = pd.concat([raw_data,nutrients_normalized], axis=1)
raw_data = raw_data.replace(np.nan,0)

raw_data.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


In [125]:
X = raw_data.values
X_ingredients = ingredients_normalized.values
X_nutrients = nutrients_normalized.values
y = label_data.values
all_data = label_data.join(raw_data)

for i in range(276):
  label = label_data['Classification'][i].rstrip()
  if label.startswith('LATICÍNIOS'): label = label.replace('LATICÍNIOS', 'DAIRY') 
  elif label.startswith('CARNES'): label = label.replace('CARNES', 'MEAT') 
  elif label.startswith('FRANGOS'): label = label.replace('FRANGOS', 'POULTRY')
  elif label.startswith('SUÍNOS'): label = label.replace('SUÍNOS', 'PORK')
  elif label.startswith('PESCADOS'): label = label.replace('PESCADOS', 'FISH')
  else: label = label.replace('OVOS', 'EGG')
  label_data.at[i,'Classification'] = label
  all_data.at[i,'Classification'] = label

print(all_data['Classification'].value_counts())

DAIRY 1      80
DAIRY 3      45
MEAT 1       34
MEAT 3       18
POULTRY 1    14
EGG 1        14
POULTRY 3    13
PORK 1       12
DAIRY 2      11
MEAT 2        9
FISH 1        8
PORK 2        6
POULTRY 2     5
FISH 2        5
EGG 2         2
Name: Classification, dtype: int64


# Visualization Techniques

### t-SNE

In [141]:
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsne = tsne.fit_transform(X)

X_tsne[1:4, :]
all_data['x'] = X_tsne[:,0]
all_data['y'] = X_tsne[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [142]:
data_ingredients = label_data.join(ingredients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsneI = tsne.fit_transform(X_ingredients)

X_tsneI[1:4, :]
data_ingredients['x'] = X_tsneI[:,0]
data_ingredients['y'] = X_tsneI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [143]:
data_nutrients = label_data.join(nutrients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsneN = tsne.fit_transform(nutrients_normalized.values)

X_tsneN[1:4, :]
data_nutrients['x'] = X_tsneN[:,0]
data_nutrients['y'] = X_tsneN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

### UMAP

In [144]:
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umap = reducer.fit_transform(X)

X_umap[1:4, :]
all_data['x'] = X_umap[:,0]
all_data['y'] = X_umap[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [145]:
data_ingredients = label_data.join(ingredients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umapI = reducer.fit_transform(X_ingredients)

X_umapI[1:4, :]
data_ingredients['x'] = X_umapI[:,0]
data_ingredients['y'] = X_umapI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [146]:
data_nutrients = label_data.join(nutrients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=20,metric='euclidean')
X_umapN = reducer.fit_transform(X_nutrients)

X_umapN[1:4, :]
data_nutrients['x'] = X_umapN[:,0]
data_nutrients['y'] = X_umapN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

### PCA

In [147]:
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

X_pca[1:4, :]
all_data['x'] = X_pca[:,0]
all_data['y'] = X_pca[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [148]:
data_ingredients = label_data.join(ingredients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pcaI = sklearn_pca.fit_transform(X_ingredients)

X_pcaI[1:4, :]
data_ingredients['x'] = X_pcaI[:,0]
data_ingredients['y'] = X_pcaI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [149]:
data_nutrients = label_data.join(nutrients_normalized)
sklearn_pca = sklearnPCA(n_components=2)
X_pcaN = sklearn_pca.fit_transform(X_nutrients)

X_pcaN[1:4, :]
data_nutrients['x'] = X_pcaN[:,0]
data_nutrients['y'] = X_pcaN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

### ISOMAP

In [150]:
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomap = isomap.fit_transform(X)

X_isomap[1:4, :]
all_data['x'] = X_isomap[:,0]
all_data['y'] = X_isomap[:,1]

fig = px.scatter(all_data, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [151]:
data_ingredients = label_data.join(ingredients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomapI = isomap.fit_transform(X_ingredients)

X_isomapI[1:4, :]
data_ingredients['x'] = X_isomapI[:,0]
data_ingredients['y'] = X_isomapI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

In [152]:
data_nutrients = label_data.join(nutrients_normalized)
isomap = Isomap(n_components=2,n_neighbors=10)
X_isomapN = isomap.fit_transform(X_nutrients)

X_isomapN[1:4, :]
data_nutrients['x'] = X_isomapN[:,0]
data_nutrients['y'] = X_isomapN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Product's Name"])
fig.show()

# Trustworthiness

### t-SNE

In [93]:
kneigh = []
y_tsne = np.zeros((10,21))
y_pca = []
y_isomap = []

v_perplexity = [5,10,15,20,25,30,35,40,45,50]
v_learning_rate = [10,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]

for pv in range(0,21):
  kneigh.append(v_learning_rate[pv])
  for pi in range(0,10):
    tsne = TSNE(n_components=2,perplexity=v_perplexity[pi],learning_rate=v_learning_rate[pv],metric='euclidean', init='random')
    X_tsne = tsne.fit_transform(X)
    precision = trustworthiness(X, X_tsne, n_neighbors=1, metric='euclidean')
    y_tsne[pi][pv] = precision 

In [94]:
fig = go.Figure()

for pi in range(0,10):
  fig.add_trace(go.Scatter(x=kneigh, y=y_tsne[pi],
                    mode='lines+markers',
                    name='t-SNE perplexity ' + str(v_perplexity[pi])))

fig.show()

### UMAP

In [95]:
v_neighbor = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_umap = []

for n in v_neighbor:
  kneigh.append(n)
  reducer = umap.UMAP(n_components=2,n_neighbors=n,metric='euclidean')
  X_umap = reducer.fit_transform(X)
  precision = trustworthiness(X, X_umap, n_neighbors=1, metric='euclidean')
  y_umap.append(precision)

In [96]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

### ISOMAP

In [97]:
v_neighbor = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_isomap = []

for n in v_neighbor:
  kneigh.append(n)
  isomap = Isomap(n_components=2,n_neighbors=n,)
  X_isomap = isomap.fit_transform(X)
  precision = trustworthiness(X, X_isomap, n_neighbors=1, metric='euclidean')
  y_isomap.append(precision)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_isomap.py:324: UserWarning:

The number of connected components of the neighbors graph is 27 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [98]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

### Neighborhood Preservation

All Data (Ingredients + Nutrients)

In [ ]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsne, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umap, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pca, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomap, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

Only Ingredients

In [ ]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsneI, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umapI, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pcaI, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomapI, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

Only Nutrients

In [ ]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsneN, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umapN, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pcaN, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomapN, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

# Silhouette Coefficient

### t-SNE

In [167]:
clusterer = KMeans(n_clusters=10, random_state=10)

cluster_labels_high = clusterer.fit_predict(X)
silhouette_avg_high = silhouette_score(X, cluster_labels_high)

cluster_labels_low = clusterer.fit_predict(X_tsne)
silhouette_avg_low = silhouette_score(X_tsne, cluster_labels_low)

cluster_labels_lowI = clusterer.fit_predict(X_tsneI)
silhouette_avg_lowI = silhouette_score(X_tsneI, cluster_labels_lowI)

cluster_labels_lowN = clusterer.fit_predict(X_tsneN)
silhouette_avg_lowN = silhouette_score(X_tsneN, cluster_labels_lowN)

print(silhouette_avg_high)
print(silhouette_avg_low)
print(silhouette_avg_lowI)
print(silhouette_avg_lowN)

0.0869328651249219
0.4174194
0.3849945
0.5282677


### UMAP

In [168]:
clusterer = KMeans(n_clusters=10, random_state=10)

cluster_labels_high = clusterer.fit_predict(X)
silhouette_avg_high = silhouette_score(X, cluster_labels_high)

cluster_labels_low = clusterer.fit_predict(X_umap)
silhouette_avg_low = silhouette_score(X_umap, cluster_labels_low)

cluster_labels_lowI = clusterer.fit_predict(X_umapI)
silhouette_avg_lowI = silhouette_score(X_umapI, cluster_labels_lowI)

cluster_labels_lowN = clusterer.fit_predict(X_umapN)
silhouette_avg_lowN = silhouette_score(X_umapN, cluster_labels_lowN)

print(silhouette_avg_high)
print(silhouette_avg_low)
print(silhouette_avg_lowI)
print(silhouette_avg_lowN)

0.0869328651249219
0.4441433
0.45596868
0.6450109


### PCA

In [170]:
clusterer = KMeans(n_clusters=10, random_state=10)

cluster_labels_high = clusterer.fit_predict(X)
silhouette_avg_high = silhouette_score(X, cluster_labels_high)

cluster_labels_low = clusterer.fit_predict(X_pca)
silhouette_avg_low = silhouette_score(X_pca, cluster_labels_low)

cluster_labels_lowI = clusterer.fit_predict(X_pcaI)
silhouette_avg_lowI = silhouette_score(X_pcaI, cluster_labels_lowI)

cluster_labels_lowN = clusterer.fit_predict(X_pcaN)
silhouette_avg_lowN = silhouette_score(X_pcaN, cluster_labels_lowN)

print(silhouette_avg_high)
print(silhouette_avg_low)
print(silhouette_avg_lowI)
print(silhouette_avg_lowN)

0.0869328651249219
0.44571990406692436
0.5029809661301383
0.4201986643536571


### ISOMAP

In [169]:
clusterer = KMeans(n_clusters=10, random_state=10)

cluster_labels_high = clusterer.fit_predict(X)
silhouette_avg_high = silhouette_score(X, cluster_labels_high)

cluster_labels_low = clusterer.fit_predict(X_isomap)
silhouette_avg_low = silhouette_score(X_isomap, cluster_labels_low)

cluster_labels_lowI = clusterer.fit_predict(X_isomapI)
silhouette_avg_lowI = silhouette_score(X_isomapI, cluster_labels_lowI)

cluster_labels_lowN = clusterer.fit_predict(X_isomapN)
silhouette_avg_lowN = silhouette_score(X_isomapN, cluster_labels_lowN)

print(silhouette_avg_high)
print(silhouette_avg_low)
print(silhouette_avg_lowI)
print(silhouette_avg_lowN)

0.0869328651249219
0.5408297667778865
0.4794880746036478
0.5404837088543618


# Global Score


### t-SNE

In [130]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_tsne)
print(f"tSNE global score {gs} - All Data")

tSNE global score 0.9279990893279472 - All Data


In [131]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_tsneI)
print(f"tSNE global score {gs} - Only Ingredients")

tSNE global score 0.9128927910144622 - Only Ingredients


In [132]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_tsneN)
print(f"tSNE global score {gs} - Only Nutrients")

tSNE global score 0.9093185598465772 - Only Nutrients


### UMAP

In [153]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_umap)
print(f"UMAP global score {gs} - All Data")

UMAP global score 0.9584380568012754 - All Data


In [154]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_umapI)
print(f"UMAP global score {gs} - Only Ingredients")

UMAP global score 0.9347293669754816 - Only Ingredients


In [155]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_umapN)
print(f"UMAP global score {gs} - Only Nutrients")

UMAP global score 0.9276850193828419 - Only Nutrients


### PCA

In [160]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_pca)
print(f"PCA global score {gs} - All Data")

PCA global score 0.9999999998173376 - All Data


In [158]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_pcaI)
print(f"PCA global score {gs} - Only Ingredients")

PCA global score 0.9979536472714037 - Only Ingredients


In [159]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_pcaN)
print(f"PCA global score {gs} - Only Nutrients")

PCA global score 0.9343260705786325 - Only Nutrients


### ISOMAP

In [161]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_isomap)
print(f"ISOMAP global score {gs} - All Data")

ISOMAP global score 0.952801971280963 - All Data


In [162]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_isomapI)
print(f"ISOMAP global score {gs} - Only Ingredients")

ISOMAP global score 0.9489105179914037 - Only Ingredients


In [163]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_isomapN)
print(f"ISOMAP global score {gs} - Only Nutrients")

ISOMAP global score 0.9276450320713224 - Only Nutrients
